### Введение
С развитием LLM-моделей спрашивать теоретическую часть или короткие алгоритмы и принципы стало бесполезно, поэтому задание будет одно, но комплексное и приближенное к реальности.

Вы можете использовать любые LLM-модели и Copilot для написания кода, так как в реальной работе сотрудники отдела также имеют доступ ко всем современным инструментам (Sonnet 3.5, GPT-4o, GitHub Copilot и т. п.).

В задании вы будете работать с тестовыми данными по банкоматам и API ЦБ РФ для получения ключевой ставки.

---

### Специфика данных fees
`fees` — тарифы за обслуживание банкомата инкассаторами. Так как банкоматы могут находиться в разных регионах, тарифы у них могут отличаться.

- **CashDeliveryFixedFee** — фиксированная стоимость доставки наличных (не зависит от суммы).
- **CashDeliveryPercentageFee** — процент от суммы доставленных наличных. Он добавляется к фиксированной стоимости. Если в таблице указано `0.0001`, это значит 0.01% от суммы.
- **CashDeliveryMinFee** — минимальная сумма, которую с нас возьмут по `CashDeliveryPercentageFee`. Например, если мы попросим инкассаторов в ATM_4 довезти всего 10 000 рублей, то с нас возьмут 5250 рублей (фиксированная стоимость), а так как `10 000 * 0.04% < 450 рублей`, то дополнительно возьмут ещё 450 рублей. Итог: 5250 + 450 рублей.

**CashCollection** — это пример сложного тарифа, который учитывает специфику работы с банкоматом. В банкомат деньги не докладывают, а меняют сразу кассету. Есть провайдеры, которые тарифицируют не только доставку, но и пересчёт денег в извлечённой кассете. Например, в ATM_4 у нас именно такой тариф.  
Пример: у нас в банкомате осталось 500 тыс. рублей, а мы хотим, чтобы у него был баланс 2 млн. Тогда мы заказываем довезти кассету на 2 млн. и платим по тарифу за доставку. При доставке старая кассета извлекается, и мы также по тарифу платим за пересчёт 500 тыс. рублей (0.45%, но не менее 1140 рублей).

- **CashCollectionFixedFee** — фиксированная стоимость за извлечение старой кассеты (в данных примерах нулевая).
- **CashCollectionPercentageFee** — процент от суммы извлечённой кассеты.
- **CashCollectionMinFee** — минимальная сумма, которую с нас возьмут по `CashCollectionPercentageFee`.

---

### Специфика данных transactions
Таблица содержит данные по снятиям, пополнениям (инкассациям) и балансу банкомата на конец дня.  
Считаем, что банкоматы в начале года пустые и не работали, поэтому баланс на конец дня равен 0, пока не случится первая инкассация.

- **bal_end_of_day** — баланс на конец дня.
- **cash_in** — пополнение в результате инкассации.
- **cash_out** — снятие наличных клиентами.

---

In [9]:
import pandas as pd

# Забираем данные по тарфиам на обслуживание банкоматов
fees = pd.read_parquet('https://storage.yandexcloud.net/norvpublic/fees.parquet')
# статистика операция по дням.
transactions = pd.read_parquet('https://storage.yandexcloud.net/norvpublic/transactions.parquet')

pd.options.display.max_columns = None

print("\nfees:\n", fees, "\n\n")
print("transactions:\n", transactions, "\n")

### Часть 1 — упущенный процентный доход

Специалисту по ML важно уметь получать данные с различных API и читать документацию. Для расчёта упущенного процентного дохода нужно обратиться к API ЦБ РФ и получить динамику ключевой ставки за 2024 год.

https://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx?op=KeyRate

Для запроса к API не нужен токен и регистрация. ЦБ РФ для части данных предпочитает использовать SOAP.

Учитывая, что хранить наличные деньги в банкомате — не самое удачное инвестиционное решение, посчитайте упущенный процентный доход для каждого банкомата. Рассчитываем, что банк мог бы вложить эти деньги и получить доход, равный ключевой ставке ЦБ РФ, актуальной на день баланса банкомата. Добавьте к таблице `transactions` столбец с упущенной процентной выгодой.


In [ ]:
import requests
import xml.etree.ElementTree as ET

def get_rate(date):

    url = "https://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx"

    SOAPEnvelope = f"""
    <soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
    <soap:Body>
        <KeyRate xmlns="http://web.cbr.ru/">
        <fromDate>{date}</fromDate>
        <ToDate>{date}</ToDate>
        </KeyRate>
    </soap:Body>
    </soap:Envelope>"""

    options = {"Content-Type": "text/xml; charset=utf-8"}

    response = requests.post(url, data=SOAPEnvelope, headers=options)
    root = ET.fromstring(response.text)

    for i in root.iter("Rate"):
        return float(i.text)

transactions["date"] = transactions["date"].dt.date

transactions = transactions.assign(Rate=transactions["date"].apply(get_rate))

print(transactions)

### Часть 2 - расходы на инкассацию

Теперь к данным по банкоматам, кроме упущенного процентного дохода, необходимо добавить столбец расходов на инкассацию. Для простоты считаем, что мы не платим за аренду помещения и страхование, поэтому нам достаточно данных по тарифам на инкассацию (`fees`).

In [ ]:
transactions = (transactions.set_index("ATM_ID").join(fees.set_index("ATM_ID")).iloc[0:, :6])

transactions = transactions.assign(result=transactions['bal_end_of_day'] >= transactions['cash_out'])

### Часть 3 - анализ данных

Посмотрите на данные о снятиях, инкассациях и балансе. В данных подобраны банкоматы, отличающиеся по характеру использования и частоте инкассаций.  
(Эту операцию вы делаете для себя, чтобы лучше понять специфику.)

In [ ]:
multi_agg = {
    'bal_end_of_day': ['mean', 'median', 'sum'],
    'cash_in': ['mean', 'sum', 'max'],
    'cash_out': ['mean', 'sum', 'max'],
    'Rate': ['median']
}

print(transactions.groupby(pd.Grouper(key='date', freq='M')).agg(multi_agg).round(1))

print(transactions.groupby('ATM_ID').agg(multi_agg).round(1))

### Часть 4 - меняем бизнес с помощью ML

Заключительная и самая интересная часть. К данному этапу у нас уже рассчитан упущенный процентный доход и расходы на инкассацию. Подумайте, как использовать ваши знания, чтобы оптимизировать процесс инкассации и уменьшить издержки.

Для данной задачи считаем, что нас устроит Service Level на уровне 90%. Это значит, что если в 9 из 10 случаев наши клиенты получают нужные им суммы, нас это устраивает.

Напоминаем, что технически в банкомат нельзя довнести сумму, и кассета меняется полностью. Соответственно, с нас берут оплату за полную кассету (если вдруг решите подойти к задаче через классическую формулу EOQ).

Вы можете подойти к задаче абсолютно любым способом и использовать все возможности ООП, Python и любых библиотек. Всё как в реальной работе, где вас никто не ограничивает.

Эффективность своего решения вы можете показать с помощью ретротестирования. Считаем, что каждая кассета может вмещать абсолютно любые суммы. Чтобы добавить реализма, вы можете считать, что деньги нужно заказывать за 3 дня до их доставки.


Представьте, что итог вашей работы состоит из 2 частей:
1) Вы презентуете своё решение руководителям банка. В этой части вы показываете, как ваше решение экономит деньги. Вы можете показать сценарий с Service Level 90% как базовый и сравнить, сколько мы сэкономим денег, если снизим его до 80% или увеличим до 95%.
2) Вы презентуете свою систему для исполнителей, а им нужно понимать за 3 дня до доставки кассеты, какую сумму заказывать, какой баланс у банкомата в какие даты должен быть. Возможно, решите ещё что-то им предоставить из данных, чтобы исполнители эффективно реализовали вашу задумку.


In [ ]:
transactions = transactions.assign(result=transactions['bal_end_of_day'] >= transactions['cash_out'])

print(round(transactions['result'].sum()/len(transactions),2))

# В данный момент общий Service Level = 99%

transactions = transactions.loc[((transactions['bal_end_of_day'] > 0))]

transactions.to_excel('transactions.xlsx')

# Выберем значения с ненулевым балансом и сохраним в файл

transactions = pd.read_excel('transactions.xlsx')

for i in range(4):
    transactions.loc[((transactions['ATM_ID'] == f'ATM_{i+1}'))].to_excel(f'ATM_{i+1}.xlsx', index=False)
    
# Сохраним значения по каждому терминалу в отдельный файл, поскольку сранивать их разумнее отдельно

ATM_1 = pd.read_excel('ATM_1.xlsx')
print(round(ATM_1['result'].sum()/len(ATM_1),2))

ATM_2 = pd.read_excel('ATM_2.xlsx')
print(round(ATM_2['result'].sum()/len(ATM_2),2))

ATM_3 = pd.read_excel('ATM_3.xlsx')
print(round(ATM_3['result'].sum()/len(ATM_3),2))

ATM_4 = pd.read_excel('ATM_4.xlsx')
print(round(ATM_4['result'].sum()/len(ATM_4),2))

# В данный момент частные показатели Service Level: 99%, 97%, 100%, 100%

# Изчуив данные, мне кажется, задача заключается в поиске минимума разницы между комиссией службе доставки и процентов от лежащей в терминале доставленной суммы

# Мои вычисления по каждому терминалу см в файле ATM_N.

# Сравнение с разными сценарими Servise Level на листе "Сравнение" в файле ATM_N.


### Заключение

Решение вы можете предоставить любым способом: репозиторий GitHub, Google Colab, конвертированный IPython Notebook в PDF и т. п.

По срокам вас не ограничивают, но учитывайте, что в это же время задачу могут решать и другие кандидаты. Таким образом, вы участвуете в конкурсе с ними.